**Автор:** Сергей Шмаков

**Обновление этого документа** и не только публикуется на канале в Телеграм - https://telegram.me/smm_python

Другие мои площадки:
* ВКонтакте - https://vk.com/smmblog
* Youtube - https://youtube.com/smmblog
* Блог - http://freesmm.ru

# Урок 5: анализ всех постов в группе ВКонтакте

**запустите код ниже**

In [1]:
from IPython.display import YouTubeVideo
YouTubeVideo("NMatAWwa7Ac")

> из этого урока вы узнаете как можно получить все посты из группы или профиля ВКонтакте

В уроке №4 мы разобрали как обрабатывать данные, чтобы в дальнейшем с ними работать. 

В 4 уроке у нас получилось получить данные только о 100 постах за раз, так как метод **wall.get** (https://vk.com/dev/wall.get) за один раз отдать больше не может. НО!

В методе есть параметр **offset**, который означает смещение.Это означает, что мы можем получать данные от первого до последнего поста пачками по 100 штук.

Упрощённая версия без обработки данных

**запустите код ниже**

In [ ]:
import requests
r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':-38369814,'count':3, 'offset':0}) 
                                                        #без смещения - получим 3 последних поста - 1,2,3
print(r.json()) #выводим на экран первые ответ
r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':-38369814,'count':3, 'offset':3})
                                                        #смещаемся от последнего поста на 3 - получаем 4,5,6 пост
print(r.json()) #выводим на экран второй ответ

Давайте теперь подумаем и попытаемся нарисовать алгоритм, который нам нужно описать, чтобы получить столько постов, сколько будет на стене

Ограничения:
* за один запрос мы можем получить до 100 постов
* максимальное смещение 100
* запросы к ВК отправлять не чаще 3 запросов в секунду

>прежде чем начинать писать, подумайте в каком порядке нужно запускать определенные действия. Попробуйте нарисовать. Проще картинки править, чем непонятный код, который вы не понимаете.

Алгоритм:
0. Смещение больше или равно количеству публикаций? Если нет, то п.1 \ Если да, то все посты обработаны
1. Отправляем запрос к ВК для получения последних 100 записей
2. Обрабатываем полученные данные (см. урок 4) и выводим на экран
3. Увеличиваем смещение на 100 и возвращаемся к 1 пункту

Картинка алгоритма
![alt](https://api.monosnap.com/rpc/file/download?id=Q3YXzb6SK9lu1XoPp6TUiWGbQWHGE7)

**Запустите код ниже**
он позволяет получить информацию о количестве лайков, репостов, комментариев в любом сообществе или профиле

**Не запускайте этот код для профилей** и сообществ, в которых много постов. Для сообществ и профилей с большим количеством постов лучше сразу писать в файл. Код ниже для демострации логики процесса

In [ ]:
import requests
import time

group_id = -38369814 #ID сообществ нужно писать с минусом, профиля - без
count = 100 #количество постов, получаемых за 1 запрос
offset = 0 #начальное смещение

#Узнаем сколько постов в сообществе

r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':group_id,'count':0})
response = r.json()
post_count = response['response'][0] #количество постов в группе или профиле
print('Количество постов в сообществе:',post_count)

print('Дата публикации',',','Ссылка',',','Количество лайков',',','Количество репостов',',','Количество комментариев')

while offset < post_count:
    r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':group_id,'count':count, 'offset':offset})
    response = r.json()

    #запускаем цикл для обработки ответа
    for i in range(1,101):
        post_date = response['response'][i]['date'] #дата и время публикации в Unix-формате
        comments_count = response['response'][i]['comments']['count'] #количество комментариев
        post_id = response['response'][i]['id'] #id поста в сообществе
        group_id = response['response'][i]['to_id'] #id сообщества . В версиях API ниже 5.7 вместо поля owner_id приходит to_id.
        likes_count = response['response'][i]['likes']['count'] #количество лайков
        repost_count = response['response'][i]['reposts']['count']  #количество репостов
        print(post_date,',','https://vk.com/wall'+str(group_id)+'_'+str(post_id),',', 
              likes_count,',',repost_count,',',comments_count)
    offset = offset + 100
    time.sleep(.33)

## Продолжение 5 урока
>7 мая 2016 г. Продолжение урока 5

В 5 уроке мы получили "странную" дату в unix-формате. 

### Теория
* unix-время: https://ru.wikipedia.org/wiki/UNIX-%D0%B2%D1%80%D0%B5%D0%BC%D1%8F
* онлайн-конвертер: http://www.cy-pr.com/tools/time/

Для её преобразования нужно использовать модуль **datetime** - http://pythonworld.ru/moduli/modul-datetime.html и **time** - http://pythonworld.ru/moduli/modul-time.html

Так в примере выше мы получаем дату и время поста от ВКонтакте

```python
post_date = response['response'][i]['date'] #дата и время публикации в Unix-формате
```

Он возвращает нам нечто вроде 1459255942.

> Задача: преобразовать 1459255942 в формам времени - Год-Месяц Час-Минута-Секунда

**запустите код ниже**

In [6]:
import datetime #импортируем пакеты
import time #импортируем пакеты

unix_time = 1459255942 #время в unix-формате

normal_time =  datetime.datetime.fromtimestamp( #функция преобразования
                int(1459255942)
                ).strftime('%Y-%m-%d %H:%M:%S') #фортма преобразования Год-Месяц Час-Минута-Секунда

print('Дата публикации:',normal_time)

Дата публикации: 2016-03-29 15:52:22


Теперь код для получения списка постов с количество лайков\репостов\комментариев и **нормальной** датой будет выглядеть так:

**запустите код ниже**

In [2]:
import requests
import datetime #импортируем пакеты
import time #импортируем пакеты

group_id = -38369814 #ID сообществ нужно писать с минусом, профиля - без
count = 100 #количество постов, получаемых за 1 запрос
offset = 0 #начальное смещение

#Узнаем сколько постов в сообществе

r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':group_id,
                                                              'count':0})
response = r.json()
post_count = response['response'][0] #количество постов в группе или профиле
print('Количество постов в сообществе:',post_count)

print('Дата публикации',',','Ссылка',',','Количество лайков',',','Количество репостов',',','Количество комментариев')

while offset < post_count:
    r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':group_id,'count':count, 'offset':offset})
    response = r.json()

    #запускаем цикл для обработки ответа
    for i in range(1,101):
        post_date =  datetime.datetime.fromtimestamp( #функция преобразования
                int(response['response'][i]['date'])
                ).strftime('%Y-%m-%d') #фортма преобразования Год-Месяц Час-Минута-Секунда   
        post_date_h =  datetime.datetime.fromtimestamp( #функция преобразования
                int(response['response'][i]['date'])
                ).strftime('%H:%M:%S') #фортма преобразования Год-Месяц Час-Минута-Секунда 
        comments_count = response['response'][i]['comments']['count'] #количество комментариев
        post_id = response['response'][i]['id'] #id поста в сообществе
        group_id = response['response'][i]['to_id'] #id сообщества . В версиях API ниже 5.7 вместо поля owner_id приходит to_id.
        likes_count = response['response'][i]['likes']['count'] #количество лайков
        repost_count = response['response'][i]['reposts']['count']  #количество репостов
        print(post_date, ',', post_date_h,',','https://vk.com/wall'+str(group_id)+'_'+str(post_id),',', 
              likes_count,',',repost_count,',',comments_count)
    offset = offset + 100
    time.sleep(.33)

Количество постов в сообществе: 2385
Дата публикации , Ссылка , Количество лайков , Количество репостов , Количество комментариев
2016-05-18 , 10:12:25 , https://vk.com/wall-38369814_19731 , 60 , 20 , 1
2016-05-20 , 19:22:58 , https://vk.com/wall-38369814_19747 , 10 , 0 , 0
2016-05-19 , 12:18:47 , https://vk.com/wall-38369814_19739 , 34 , 4 , 0
2016-05-18 , 17:51:48 , https://vk.com/wall-38369814_19737 , 21 , 2 , 1
2016-05-18 , 16:21:44 , https://vk.com/wall-38369814_19736 , 15 , 2 , 0
2016-05-18 , 11:11:48 , https://vk.com/wall-38369814_19733 , 5 , 0 , 0
2016-05-17 , 15:36:20 , https://vk.com/wall-38369814_19711 , 25 , 3 , 8
2016-05-17 , 09:17:16 , https://vk.com/wall-38369814_19708 , 31 , 10 , 2
2016-05-16 , 10:05:16 , https://vk.com/wall-38369814_19690 , 7 , 1 , 2
2016-05-15 , 18:58:16 , https://vk.com/wall-38369814_19677 , 54 , 20 , 23
2016-05-13 , 18:36:36 , https://vk.com/wall-38369814_19663 , 11 , 0 , 5
2016-05-13 , 13:34:47 , https://vk.com/wall-38369814_19650 , 25 , 1 , 11
201

IndexError: list index out of range

## Итог 4 и 5 уроков

**Чему мы научились:**
1. обрабатывать ответ от ВКонтакте так, как нужно нам!
2. запускать цикл в цикле для получения множества объектов (смещение по **offset**, а затем:обработка ответа)
3. преобразовывать дату из unix-формата в тот, который нужен нам

Таким образом мы написали универсальный скрипт для анализа любого открытого профиля и сообщества, из которых мы можем получить список всех публикаций с данными о каждой публикации:
* дата публикации
* количество "Мне нравится"
* количество "Рассказать друзьям"
* Количество комментариев

### Домашнее задание к 4 и 5 уроку
> Получить **дату публикации** и **автора** из конкретного обсуждения. Возьмём для примера https://vk.com/topic-38369814_28008826

Ответы и вопросы по домашке публикуйте здесь - https://vk.com/topic-38369814_33529150 